In [77]:
# from ipynb.fs.defs.Subreddit_Embedding_Temporal import load_embedding, parse_tup, to_date
from utils import load_embedding, parse_tup, cos_sim
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

In [78]:
TIME_FRAME = "biweekly"
subreddits, vectors  = load_embedding("/h/224/cameron/Political-Subreddit-Embedding/trained_embeddings/vecs_0.0043_35.txt")
# # PCA Dim Reduction
# pca =  PCA(n_components = 2)
# vectors =  pd.DataFrame(pca.fit_transform(vectors))
embedding = pd.concat([subreddits, vectors], axis=1).reset_index(drop=True)
del subreddits, vectors
embedding

,subreddit,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,SquaredCircle,-0.048178,0.021482,-0.033970,-0.061168,-0.029943,0.090738,-0.060890,-0.064819,0.046365,...,-0.080529,0.009409,0.090371,0.160984,-0.012692,0.037296,-0.133719,-0.054398,0.045666,-0.094102
1,AskReddit,0.039633,-0.130123,0.047644,0.073585,0.089806,0.015975,-0.077038,-0.023020,0.097915,...,-0.003924,0.094406,-0.028855,0.182672,0.114283,0.046218,-0.122043,-0.027566,0.101160,-0.094482
2,politics,0.014866,0.105574,-0.015800,-0.009825,0.059193,0.157330,-0.094050,-0.165119,0.077472,...,-0.101872,0.055519,0.001593,0.089291,0.035413,0.046253,-0.087247,0.085222,0.025805,-0.159948
3,memes,0.051227,-0.011974,0.003561,0.047713,0.004941,-0.053547,-0.138880,0.075173,-0.014687,...,-0.056815,0.006998,0.090546,0.055618,-0.003918,-0.102020,-0.056633,0.013987,-0.081270,-0.055979
4,teenagers,-0.095111,0.053749,0.011059,-0.072698,-0.062629,-0.075457,-0.029677,0.017484,-0.016888,...,0.071732,0.082159,-0.182717,0.127624,0.015954,-0.206583,-0.084462,-0.089431,0.066405,-0.076050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,SocialDemocracy,-0.023875,-0.024923,-0.013402,0.003846,0.006347,0.069092,-0.079145,-0.027321,0.018481,...,0.104024,0.098131,-0.065208,0.146092,0.105407,0.027191,-0.073014,-0.022048,-0.021949,-0.049449
1006,BaemyKlobaechar,0.009682,-0.052802,0.037955,-0.115337,-0.053914,0.084777,-0.156884,-0.027475,-0.084624,...,0.039904,0.116992,-0.021720,0.167901,0.106617,0.094536,-0.032064,-0.040523,0.008024,-0.004132
1007,demsocialist,-0.007730,-0.040278,-0.072353,0.011064,0.006521,0.090017,-0.112426,-0.012652,-0.025386,...,0.048959,0.084410,-0.015494,0.219271,0.147901,0.078930,-0.101491,-0.111678,-0.004957,-0.069073
1008,BetoORourke,0.017894,-0.040889,0.054302,0.020973,-0.067203,0.142831,-0.197403,-0.069740,-0.014712,...,-0.001153,0.058416,0.106178,0.163847,0.100974,0.032369,-0.143735,-0.036005,-0.009335,-0.056085


In [133]:
def find_closest(subreddit,transformation,k=5):
    # Get subreddit
    assert subreddit in embedding.subreddit.values, "{} is not in the dataset".format(subreddit)
    sub_vec = embedding.loc[embedding["subreddit"] == subreddit].drop("subreddit",axis=1).to_numpy().flatten()
    assert sub_vec.shape == transformation.shape, "Transformation vector dimensions {} do not equal embedding dimensions {}".format(transformation.shape,sub_vec.shape)
    transformed = sub_vec + transformation
    all_other_vecs = embedding.drop(["subreddit"],axis=1).to_numpy()
    similarity = np.apply_along_axis(lambda x : cos_sim(transformed,x), 1, all_other_vecs)
    similarity = pd.DataFrame({"subreddit": embedding["subreddit"], "similarity": similarity}).sort_values("similarity",ascending=False)
    similarity = similarity[similarity["subreddit"] != subreddit]


def analogy(sub_a1,sub_b1,sub_a2):
    """
        sub_a1 is to sub_b1, as sub_a2 is to ??
        sub_a2 - sub_a1 + sub_b1
        returns the subreddit that best answers the analogy
    """
    assert sub_a1 in embedding.subreddit.values, "{} is not in the dataset".format(sub_a1)
    assert sub_b1 in embedding.subreddit.values, "{} is not in the dataset".format(sub_b1)
    assert sub_a2 in embedding.subreddit.values, "{} is not in the dataset".format(sub_a2)
    a1vec = embedding.loc[embedding["subreddit"] == sub_a1].drop("subreddit",axis=1).to_numpy().flatten()
    b1vec = embedding.loc[embedding["subreddit"] == sub_b1].drop("subreddit",axis=1).to_numpy().flatten()
    a2vec = embedding.loc[embedding["subreddit"] == sub_a2].drop("subreddit",axis=1).to_numpy().flatten()
    b2vec = a2vec - a1vec + b1vec
    all_other_vecs = embedding.drop(["subreddit"],axis=1).to_numpy()
    similarity = np.apply_along_axis(lambda x : cos_sim(x,b2vec), 1, all_other_vecs)
    similarity = pd.DataFrame({"subreddit": embedding["subreddit"], "similarity": similarity}).sort_values("similarity",ascending=False)
    similarity = similarity[~similarity["subreddit"].isin([sub_a1,sub_b1,sub_a2])]
    print("{} is to {} as {} is to {}".format(sub_a1,sub_b1,sub_a2,similarity["subreddit"].iloc[0]))
    return similarity["subreddit"].iloc[0]

print(analogy("ElizabethWarren","BaemyKlobaechar","SandersForPresident"))
transformation = get_transformation("demsocialist","centerleftpolitics")
find_closest("BaemyKlobaechar",transformation)

ElizabethWarren is to BaemyKlobaechar as SandersForPresident is to JoeBiden
JoeBiden
             subreddit  similarity
1007      demsocialist    0.792206
1003          JoeBiden    0.675658
1000  moderatepolitics    0.667134
1008       BetoORourke    0.661664
541          stupidpol    0.642875


In [132]:
from itertools import combinations
def predict_analogies(analogies):
    tuples = [tuple(x) for x in analogies.to_numpy()]
    pairs = list(combinations(tuples,2))
    pred = []
    actual = []
    for (a,ap),(b,bp) in pairs:
        try: 
            pred.append(analogy(a.strip(),ap.strip(),b.strip()))
            actual.append(bp)
        except:
            pass
        
    return pred,actual

analogies = pd.read_csv("/h/224/cameron/Political-Subreddit-Embedding/external_datasets/analogies/politics.csv")
pred, actual = predict_analogies(analogies)
for a,b in zip(pred,actual):
    print(a,b)

moderatepolitics is to JoeBiden as centerleftpolitics is to SocialDemocracy
                subreddit  similarity
1005      SocialDemocracy    0.597512
1006      BaemyKlobaechar    0.548601
749        Pete_Buttigieg    0.541872
957   Enough_Sanders_Spam    0.528191
1007         demsocialist    0.521890
moderatepolitics is to JoeBiden as moderatepolitics is to BaemyKlobaechar
                 subreddit  similarity
1006       BaemyKlobaechar    0.818622
1009  ConservativeDemocrat    0.783992
1005       SocialDemocracy    0.749856
990              BreadTube    0.731570
446              investing    0.721158
moderatepolitics is to JoeBiden as centerleftpolitics is to SocialDemocracy
                subreddit  similarity
1005      SocialDemocracy    0.597512
1006      BaemyKlobaechar    0.548601
749        Pete_Buttigieg    0.541872
957   Enough_Sanders_Spam    0.528191
1007         demsocialist    0.521890
moderatepolitics is to JoeBiden as centerleftpolitics is to SocialDemocracy
        

centerleftpolitics is to BaemyKlobaechar as SocialDemocracy is to ConservativeDemocrat
                 subreddit  similarity
1009  ConservativeDemocrat    0.718735
1003              JoeBiden    0.718636
541              stupidpol    0.696406
1000      moderatepolitics    0.686678
990              BreadTube    0.684172
centerleftpolitics is to BaemyKlobaechar as demsocialist is to JoeBiden
                 subreddit  similarity
1003              JoeBiden    0.675658
1000      moderatepolitics    0.667134
1008           BetoORourke    0.661664
541              stupidpol    0.642875
1009  ConservativeDemocrat    0.638308
centerleftpolitics is to BaemyKlobaechar as progressive is to moderatepolitics
             subreddit  similarity
1000  moderatepolitics    0.681947
1003          JoeBiden    0.635194
474          SeattleWA    0.626923
285          vancouver    0.610915
1008       BetoORourke    0.605682
centerleftpolitics is to BaemyKlobaechar as progressive is to moderatepolitics
     